SDES IMPLEMENTATION

KEY GENERATION

In [1]:
P10 = [3, 5, 2, 7, 4, 10, 1, 9, 8, 6]
P8 = [6, 3, 7, 4, 8, 5, 10, 9]

In [2]:
def apply_permutation(x, p):
  res = []
  for i in p:
    res.append(x[i-1])
  return res

In [3]:
def left_shift(x, places = 1):
  places %= len(x)
  return x[places:] + x[:places]

In [4]:
# key = [1, 0, 1, 0, 0, 0, 0, 0, 1, 0]
key = [1, 1, 0, 0, 0, 1, 1, 1, 1, 0]

In [5]:
key_p10 = apply_permutation(key, P10)
print(f'Key after P10 permutation: {key_p10}')

Key after P10 permutation: [0, 0, 1, 1, 0, 0, 1, 1, 1, 1]


In [6]:
l = key_p10[:5]
r = key_p10[5:]
print(f'Left half: {l}, Right half: {r}')

Left half: [0, 0, 1, 1, 0], Right half: [0, 1, 1, 1, 1]


In [7]:
l_ls1 = left_shift(l)
r_ls1 = left_shift(r)
print('Applying left shift')
print(f'Left half: {l_ls1}, Right half: {r_ls1}')

Applying left shift
Left half: [0, 1, 1, 0, 0], Right half: [1, 1, 1, 1, 0]


In [8]:
key_ls1 = l_ls1 + r_ls1
k1 = apply_permutation(key_ls1, P8)
print(f'Key-1 after applying P8 permutation: {k1}')

Key-1 after applying P8 permutation: [1, 1, 1, 0, 1, 0, 0, 1]


In [9]:
l_ls2 = left_shift(l_ls1, 2)
r_ls2 = left_shift(r_ls1, 2)
print('Applying double left shift')
print(f'Left half: {l_ls2}, Right half: {r_ls2}')

Applying double left shift
Left half: [1, 0, 0, 0, 1], Right half: [1, 1, 0, 1, 1]


In [10]:
key_ls2 = l_ls2 + r_ls2
k2 = apply_permutation(key_ls2, P8)
print(f'Key-2 after applying P8 permutation: {k2}')

Key-2 after applying P8 permutation: [1, 0, 1, 0, 0, 1, 1, 1]


In [11]:
print('Final keys obtained')
print(f'Key-1: {k1}')
print(f'Key-2: {k2}')

Final keys obtained
Key-1: [1, 1, 1, 0, 1, 0, 0, 1]
Key-2: [1, 0, 1, 0, 0, 1, 1, 1]


ENCRYPTION AND DECRYPTION

In [12]:
# plain_text = [1, 0, 0, 1, 0, 1, 1, 1]
plain_text = [0, 0, 1, 0, 1, 0, 0, 0]

In [13]:
initial_permutation = [2, 6, 3, 1, 4, 8, 5, 7]
expansion_permutation = [4, 1, 2, 3, 2, 3, 4, 1]
P4 = [2, 4, 3, 1]
inv_initial_permutation = [4, 1, 3, 5, 7, 2, 8, 6]

In [14]:
text_ip = apply_permutation(plain_text, initial_permutation)
print(f'Text after initial permutation: {text_ip}')

Text after initial permutation: [0, 0, 1, 0, 0, 0, 1, 0]


In [15]:
l_ip = text_ip[:4]
r_ip = text_ip[4:]
r_ip_ep = apply_permutation(r_ip, expansion_permutation)
print(f'Right half after explansion: {r_ip_ep}')

Right half after explansion: [0, 0, 0, 1, 0, 1, 0, 0]


In [16]:
def apply_xor(x, y):
  res = []
  for i in range(len(x)):
    res.append(x[i] ^ y[i])
  return res

In [17]:
k1_xor = apply_xor(k1, r_ip_ep)
print(f'Key-1 after XOR: {k1_xor}')

Key-1 after XOR: [1, 1, 1, 1, 1, 1, 0, 1]


In [18]:
l_xor = k1_xor[:4]
r_xor = k1_xor[4:]

In [19]:
S0 = [[1,0,3,2],
      [3,2,1,0],
      [0,2,1,3],
      [3,1,3,2]]

S1=  [[0,1,2,3],
      [2,0,1,3],
      [3,0,1,0],
      [2,1,0,3]]

In [20]:
def apply_s_box(x, s):
  r = int(f'{x[0]}{x[3]}', 2)
  c = int(f'{x[1]}{x[2]}', 2)
  val = s[r][c]

  if val == 0: return [0, 0]
  elif val == 1: return [0, 1]
  elif val == 2: return [1, 0]
  else: return [1, 1]

In [21]:
s_box_1 = apply_s_box(l_xor, S0) + apply_s_box(r_xor, S1)
print(f'Applying S-Boxes on XOR key: {s_box_1}')

Applying S-Boxes on XOR key: [1, 0, 0, 0]


In [22]:
s_box_p4 = apply_permutation(s_box_1, P4)
print(f'After applying P4: {s_box_p4}')

After applying P4: [0, 0, 0, 1]


In [23]:
p4_xor = apply_xor(l_ip, s_box_p4)
print(f'Applying XOR with left half: {p4_xor}')

Applying XOR with left half: [0, 0, 1, 1]


In [24]:
x = p4_xor + r_ip
print(f'Combining with right half: {x}')

Combining with right half: [0, 0, 1, 1, 0, 0, 1, 0]


In [25]:
x_rev = x[4:] + x[:4]
print(f'Swapping nibbles: {x_rev}')

Swapping nibbles: [0, 0, 1, 0, 0, 0, 1, 1]


In [26]:
l_x = x_rev[:4]
r_x = x_rev[4:]
r_x_ep = apply_permutation(r_x, expansion_permutation)
print(f'Right half after expansion: {r_x_ep}')

Right half after expansion: [1, 0, 0, 1, 0, 1, 1, 0]


In [27]:
k2_xor = apply_xor(r_x_ep, k2)
print(f'After XOR with Key-2: {k2_xor}')

After XOR with Key-2: [0, 0, 1, 1, 0, 0, 0, 1]


In [28]:
l_xor_2 = k2_xor[:4]
r_xor_2 = k2_xor[4:]

In [29]:
s_box_2 = apply_s_box(l_xor_2, S0) + apply_s_box(r_xor_2, S1)
print(f'Applying S-Boxes on XOR key: {s_box_2}')

Applying S-Boxes on XOR key: [1, 0, 1, 0]


In [30]:
s_box_2_p4 = apply_permutation(s_box_2, P4)
print(f'After applying P4: {s_box_2_p4}')

After applying P4: [0, 0, 1, 1]


In [31]:
y = apply_xor(l_x, s_box_2_p4) + r_x
print(f'After XOR and joining: {y}')

After XOR and joining: [0, 0, 0, 1, 0, 0, 1, 1]


In [32]:
l_y = y[:4]
r_y = y[4:]

In [33]:
r_y_ep = apply_permutation(r_y, expansion_permutation)
print(f'Right half after expansion: {r_y_ep}')

Right half after expansion: [1, 0, 0, 1, 0, 1, 1, 0]


In [34]:
r_y_xor = apply_xor(r_y_ep, k2)
print(f'After XOR with Key-2: {r_y_xor}')

After XOR with Key-2: [0, 0, 1, 1, 0, 0, 0, 1]


In [35]:
xor_l = r_y_xor[:4]
xor_r = r_y_xor[4:]

In [36]:
s_box_3 = apply_s_box(xor_l, S0) + apply_s_box(xor_r, S1)
print(f'Applying S-Boxes on XOR key: {s_box_3}')

Applying S-Boxes on XOR key: [1, 0, 1, 0]


In [37]:
s_box_3_p4 = apply_permutation(s_box_3, P4)
print(f'After applying P4: {s_box_3_p4}')

After applying P4: [0, 0, 1, 1]


In [38]:
z = apply_xor(l_x, s_box_3_p4) + r_x
print(f'After XOR and joining: {z}')

After XOR and joining: [0, 0, 0, 1, 0, 0, 1, 1]


In [39]:
cipher_text = apply_permutation(z, inv_initial_permutation)
print(f'Cipher Text: {cipher_text}')

Cipher Text: [1, 0, 0, 0, 1, 0, 1, 0]
